In [18]:
import pandas as pd
import numpy as np


In [19]:
df = pd.read_csv("../data_source/laptop_data_preprocessed.csv")
x_train = pd.read_csv("../data_split/x_train.csv")
x_test = pd.read_csv("../data_split/x_test.csv")
y_train = pd.read_csv("../data_split/y_train.csv").squeeze()
y_test = pd.read_csv("../data_split/y_test.csv").squeeze()


In [20]:
import pickle

with open("../transformers/column_transformer.pkl", "rb") as f:
    step1 = pickle.load(f)


C:\Users\kanak\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\kanak\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\kanak\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from ver

In [21]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor



models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=15),
    'KNN': KNeighborsRegressor(n_neighbors=3),
    'Decision Tree': DecisionTreeRegressor(max_depth=8),
    'SVM': SVR(kernel='rbf', C=10000, epsilon=0.1),
    'Random Forest': RandomForestRegressor(
        n_estimators=100, random_state=3, max_samples=0.5,
        max_features=0.75, max_depth=15
    ),
    'AdaBoost': AdaBoostRegressor(n_estimators=15, learning_rate=1.0),
    'Gradient Boost': GradientBoostingRegressor(n_estimators=100),
    'XGBoost': XGBRegressor(max_depth=5, learning_rate=0.5)
}


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle
import numpy as np
import pandas as pd

# Define categorical columns
categorical_cols = ['Company', 'TypeName', 'Cpu brand', 'Gpu brand', 'os']

# Define ColumnTransformer
step1 = ColumnTransformer(
    transformers=[
        ('col_tnf', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'
)

# Fit transformer on training data
step1.fit(x_train)

results = {}
best_r2 = -np.inf
best_model_name = None
best_pipe = None

for name, model in models.items():
    pipe = Pipeline([
        ('transformer', step1),
        ('model', model)
    ])
    
    # Train model
    pipe.fit(x_train, y_train)
    
    # Predict
    y_pred = pipe.predict(x_test)
    
    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    results[name] = {'r2': r2, 'mae': mae}
    
    print(f"{name}")
    print(f"R² Score: {r2:.4f}")
    print(f"MAE Score: {mae:.4f}")
    print("-" * 50)
    
    # Check for best model
    if r2 > best_r2:
        best_r2 = r2
        best_model_name = name
        best_pipe = pipe

print("The best model effective for training is ")
best_pipe.named_steps['model']

Linear Regression
R² Score: 0.7969
MAE Score: 0.2039
--------------------------------------------------
Ridge Regression
R² Score: 0.7859
MAE Score: 0.2103
--------------------------------------------------
KNN
R² Score: 0.7710
MAE Score: 0.2115
--------------------------------------------------
Decision Tree
R² Score: 0.8085
MAE Score: 0.1952
--------------------------------------------------
SVM
R² Score: 0.8248
MAE Score: 0.1901
--------------------------------------------------
Random Forest
R² Score: 0.8646
MAE Score: 0.1638
--------------------------------------------------
AdaBoost
R² Score: 0.7727
MAE Score: 0.2240
--------------------------------------------------
Gradient Boost
R² Score: 0.8457
MAE Score: 0.1780
--------------------------------------------------
XGBoost
R² Score: 0.8708
MAE Score: 0.1572
--------------------------------------------------
The best model effective for training is 


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.5, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [23]:
results_df = pd.DataFrame(results).T.reset_index()

results_df.rename(columns={'index': 'Model Name'}, inplace=True)

results_df = results_df.sort_values(by='r2', ascending=False)

results_df.to_csv("../model_evaluation/model_performance.csv", index=False)
print("Model evaluation complete! Saved 'model_performance.csv'.")

Model evaluation complete! Saved 'model_performance.csv'.


In [24]:
pickle.dump(best_pipe, open('../models/pipe.pkl', 'wb'))
df_for_app = df.reset_index(drop=True)  # Remove any index column
pickle.dump(df_for_app, open('../models/data.pkl', 'wb'))
